In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import copy

In [ ]:
%config Completer.use_jedi = False
%matplotlib inline

In [ ]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 7)

In [ ]:
df_meta = pd.read_csv('metadata_plants.csv')

In [ ]:
df_meta.head()
# Alyrata  Csativus  Macuminata  Ppatens  Taestivum  Vcarteri
# remove test genomes
df_plants = df_meta[~df_meta['species'].isin(['Alyrata','Csativus','Macuminata','Ppatens',
                                              'Taestivum','Vcarteri'])].reset_index(drop=True)



In [ ]:
np.random.seed(11345)
x = np.repeat(range(6), 9)
np.random.shuffle(x)
x

In [ ]:
df_plants.loc[:,"random_split"] = x

In [ ]:
df_plants.loc[:,"length_split"] = -1

In [ ]:
df_plants.head()

In [ ]:
for i, split in zip(np.argsort(df_plants.loc[:, "total_len"]), np.repeat(range(6), 9)):
    df_plants.loc[i,"length_split"] = split

In [ ]:
df_plants.loc[:,"gc_split"] = -1

In [ ]:
x2 = df_plants.loc[:,"C"] / (df_plants.loc[:,"A"] + df_plants.loc[:,"C"])

In [ ]:
for i, split in zip(np.argsort(x2), np.repeat(range(6), 9)):
    df_plants.loc[i,"gc_split"] = split

In [ ]:
df_plants.loc[df_plants.loc[:, "random_split"] == 1, "species"]

In [ ]:
df_plants.loc[df_plants.loc[:, "length_split"] == 1, "species"]

In [ ]:
# and now for the phylogenetic split
remaining = list(df_plants['species'])
green_algae = ['Dsalina', 'Creinhardtii', 'Czofingiensis', 'MpusillaCCMP1545', 'MpusillaRCC299',
               'Olucimarinus', 'CsubellipsoideaC169']
remaining = [x for x in remaining if x not in green_algae]
monocots = ['Hvulgare', 'Bdistachyon', 'Osativa', 'Sbicolor', 'Zmays', 'Sitalica', 'Othomaeum', 'Acomosus',
            'Aofficinalis', 'Zmarina', 'Spolyrhiza']
remaining = [x for x in remaining if x not in monocots]

asterids = ['Hannuus', 'Lsativa', 'Dcarota', 'Mguttatus', 'Oeuropaea', 'Stuberosum',
            'Slycopersicum']
remaining = [x for x in remaining if x not in asterids]
fabids = ['Mesculenta', 'Rcommunis', 'Lusitatissimum', 'Ptrichocarpa', 'Mdomestica', 'Ppersica',
          'Fvesca', 'Mtruncatula', 'Carietinum', 'Gmax']
remaining = [x for x in remaining if x not in fabids]
malvids = ['Athaliana', 'Crubella', 'Cgrandiflora', 'Esalsugineum', 'Cpapaya', 'Graimondii', 'Csinensis',
           'Cclementina', 'Egrandis', 'Tcacao', 'Boleraceacapitata']
remaining = [x for x in remaining if x not in malvids]

remaining
df_plants.loc[:, 'phylo_split'] = 5

In [ ]:
df_plants.loc[[x in green_algae for x in df_plants.loc[:, "species"]], 'phylo_split'] = 0
df_plants.loc[[x in monocots for x in df_plants.loc[:, "species"]], 'phylo_split'] = 1
df_plants.loc[[x in asterids for x in df_plants.loc[:, "species"]], 'phylo_split'] = 2
df_plants.loc[[x in fabids for x in df_plants.loc[:, "species"]], 'phylo_split'] = 3
df_plants.loc[[x in malvids for x in df_plants.loc[:, "species"]], 'phylo_split'] = 4




In [ ]:
def mk_bash_denbi(outdir, species):
    template = """
pfx=/mnt/share/ubuntu/data/plants/single_genomes/
outdir={}

mkdir -p $outdir
python /mnt/share/ubuntu/repos/github/weberlab-hhu/helixer_scratch/data_scripts/merge-files.py \\
        --input-files {} \\
        --output-file $outdir/training_data.h5
cd $outdir
ln -s ../../eight_genomes_nosplit_phase/validation_data.h5
cd ..
"""
    
    one_path = "$pfx/{}/test_data.h5"
    sp_paths = [one_path.format(sp) for sp in species]
    return template.format(outdir, ' '.join(sp_paths))

In [ ]:
print(mk_bash_denbi('gc3', df_plants.loc[df_plants.loc[:, "gc_split"] == 3, "species"]))

In [ ]:
print(mk_bash_denbi('length5', df_plants.loc[df_plants.loc[:, "length_split"] == 5, "species"]))

In [ ]:
print(mk_bash_denbi('phylo5', df_plants.loc[df_plants.loc[:, "phylo_split"] == 5, "species"]))

In [ ]:
df_plants.loc[:, ['species', 'random_split',
       'length_split', 'gc_split', 'phylo_split']].to_csv('rabbit_splits.csv', index=False)

In [ ]:
def mergy_plotty(key, sort_by):
    # mergy
    gc_res = pd.read_csv(f'data_splits/f1_{key}.csv', header=None)
    gc_res.columns = ['model', 'species', 'sub_genic_f1', 'genic_f1']
    gc_preview = gc_res.pivot(columns='model', index='species', values='genic_f1')
    indis = gc_preview.index.to_list()
    indis = [x if (x != 'MspRCC299') else "MpusillaRCC299" for x in indis]
    gc_preview.index = indis
    x = df_plants.merge(gc_preview, left_on='species', right_index=True, how='left')
    indexes = np.argsort(x[sort_by])
    trainers = np.eye(6)[x[f'{key}_split']]
    
    # stat
    print(np.median(x.iloc[indexes, 57:63]))
    
    # plotty
    fig, (ax, bx) = plt.subplots(1,2, figsize=(4,14))
    fig.suptitle(f'{key}, sort {sort_by}', fontsize=20, y=0.93)
    y_label_list = x.loc[:,'species'][indexes]

    ax.imshow(trainers[indexes], aspect='auto', 
               interpolation='none')
    ax.set_yticks(range(54))
    ax.set_yticklabels(y_label_list)
    ax.set_xticks(range(6))
    ax.set_xlabel('split')

    img = bx.imshow(x.iloc[indexes, 57:63], aspect='auto', 
               interpolation='none')
    bx.set_xticks(range(6))
    bx.set_xlabel('split')
    bx.set_yticks(range(54))
    bx.set_yticklabels('')

    fig.colorbar(img, fraction=0.04)


    ax.set_title("trainers")
    bx.set_title("genic_f1")



In [ ]:
mergy_plotty(key='gc', sort_by='gc_content')

In [ ]:
mergy_plotty(key='length', sort_by='total_len')

In [ ]:
mergy_plotty(key='phylo', sort_by='phylo_split')

In [ ]:
mergy_plotty(key='random', sort_by='random_split')

In [ ]:
mergy_plotty(key='random', sort_by='gc_content')

In [ ]:
mergy_plotty(key='random', sort_by='total_len')

In [ ]:
mergy_plotty(key='random', sort_by='phylo_split')